<a href="https://colab.research.google.com/github/joaomarcuslf2/data-science-studies/blob/main/imersao_alura_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

fonte = "https://github.com/alura-cursos/imersao-dados-2-2020/blob/master/MICRODADOS_ENEM_2019_SAMPLE_43278.csv?raw=true"
df = pd.read_csv(fonte)
subjects = ["NU_NOTA_CN","NU_NOTA_CH","NU_NOTA_MT","NU_NOTA_LC","NU_NOTA_REDACAO"]

Desafio 01: Criar uma função que facilite plotar um gráfico

In [ ]:
# Desafio 01:
def create_plot(data, x, y, title = "", figsize = (10, 6), order = [], hue = ""):
  plt.figure(figsize=figsize)
  if (len(order) > 1):
    if (len(hue)):
      sns.boxplot(x=x, y=y, data=data, order=order, hue=hue)
    else:
      sns.boxplot(x=x, y=y, data=data, order=order)
  else:
    if (len(hue)):
      sns.boxplot(x=x, y=y, data=data, hue=hue)
    else:
      sns.boxplot(x=x, y=y, data=data)

  plt.title(title)

def create_bar_plot(data, search_key, new_key, title, xlabel, labels = [], bins = []):
  copy_bins = [-1] + bins

  data_copy = data.copy()

  data_copy[new_key] = pd.cut(
    data_copy[search_key],
    bins=copy_bins,
    labels=labels
  )

  count = data_copy[new_key].value_counts().values
  index = data_copy[new_key].value_counts().index

  plt.figure(figsize = (24, 4))
  plt.subplot(1, 2, 1)
  plt.bar(index, count, ec = "k", alpha = .6, color = "#335878")
  plt.xlabel(xlabel)
  plt.title(title)

def create_pie_plot(data, search_key, new_key, title, labels = [], bins = [], colors = []):
  copy_bins = [-1] + bins

  data_copy = data.copy()

  data_copy[new_key] = pd.cut(
    data_copy[search_key],
    bins=copy_bins,
    labels=labels
  )

  count = data_copy[new_key].value_counts().values
  index = data_copy[new_key].value_counts().index

  plt.subplot(1, 2, 2)
  plt.pie(count, 
          labels = list(index),  
          colors = colors,
          labeldistance = 1.1,
          wedgeprops = {"ec": "k"}, 
          textprops = {"fontsize": 15}, 
    )
  plt.axis("equal")
  plt.title(title)
  plt.legend()

def plot_with_x_q006(y, title = "", order = [], data = df, hue = ""):
  return create_plot(data, "Q006", y, title, (10, 6), order, hue)


In [ ]:
plot_with_x_q006(
    "NU_NOTA_MT", 
    "Boxplot das notas de matemática pela renda",
  )

In [ ]:
df_by_salary = df["Q006"].unique()
df_by_salary.sort()

plot_with_x_q006(
    "NU_NOTA_MT", 
    "Boxplot das notas de matemática pela renda ordenada",
    df_by_salary,
  )

In [ ]:
df["NU_NOTA_TOTAL"] = df[subjects].sum(axis=1)

df_by_salary = df["Q006"].unique()
df_by_salary.sort()

plot_with_x_q006(
    "NU_NOTA_TOTAL", 
    "Boxplot das notas somadas pela renda",
    df_by_salary
  )

In [ ]:
df["NU_NOTA_MEDIA"] = df[subjects].mean(axis=1)

df_by_salary = df["Q006"].unique()
df_by_salary.sort()

plot_with_x_q006(
    "NU_NOTA_MEDIA", 
    "Boxplot das notas somadas pela renda",
    df_by_salary
  )

In [ ]:
sns.displot(df, x="NU_NOTA_TOTAL")

In [ ]:
sns.displot(df, x="NU_NOTA_MEDIA")

In [ ]:
subjects.append("NU_NOTA_TOTAL")
subjects.append("NU_NOTA_MEDIA")

df.query("NU_NOTA_TOTAL == 0")[subjects]

In [ ]:
df_clean = df.query("NU_NOTA_TOTAL != 0")
df_clean.head()

In [ ]:
create_plot(
    df_clean, 
    "Q006",
    "NU_NOTA_MEDIA",
    "Boxplot das notas somadas pela renda sem inscritos zerados", 
    (14, 8),
    df_by_salary,
    "IN_TREINEIRO"
  )

Desafio 02: Verificar dados para saber quais motivos do aluno ter tirado nota 0

In [ ]:
zerados = df.query('NU_NOTA_CN == "NaN" or NU_NOTA_CN == 0')

for subject in subjects:
  if 'REDACAO' in subject or 'TOTAL' in subject or 'MEDIA' in subject:
    continue

  key = subject.replace('NU_NOTA_', 'TP_PRESENCA_')
  new_key = subject.replace('NU_NOTA_', 'TXT_PRESENCA_')

  sub = subject.replace('NU_NOTA_', '')

  dict_subs = {
      'CN': 'Ciências da Natureza',
      'CH': 'Ciências Humanas',
      'LC': 'Linguagens e Códigos',
      'MT': 'Matemática'
  }

  title = 'Motivos de notas 0 da prova de ' + dict_subs[sub]

  xlabel = 'Tipos de presenca'
  labels = [ 'Faltou à prova', 'Presente na prova', 'Eliminado na prova' ]
  bins = [ 0, 1, 2 ]
  colors = [ "#FFAD95", "#335878", "#A34858" ]

  create_bar_plot(
    zerados, 
    key, 
    new_key, 
    title, 
    xlabel, 
    labels, 
    bins,
  )

  create_pie_plot(
    zerados, 
    key, 
    new_key, 
    title, 
    labels, 
    bins,
    colors,
  )

Desafio 03: Descobrir se quem é eliminado da prova tira 0 ou NaN

Desafio 04: Verificar a proporção dos participantes de rendas mais altas e mais baixas como treineiro e não treineiro.

In [ ]:
plt.figure(figsize = (8,6))

plt.hist(df.query("IN_TREINEIRO == 1")["NU_IDADE"].value_counts().sort_index(), edgecolor = 'white', label = 'treineiro')
plt.hist(df.query("IN_TREINEIRO == 0")["NU_IDADE"].value_counts().sort_index(), edgecolor = 'white', label = 'não treineiros')

plt.legend()

plt.title('Histogramas das idades dos do treineiro e não treineiros')

plt.xlabel('Quantidade')
plt.ylabel('Idade')
plt.show()

Desafio 05: Fazer o mesmo boxplot olhando para a questão 25 (tem internet ou não) e fazer uma reflexão sobre o assunto e o contexto de pandemia.

Desafio da Thainá: Pegar a amostra completa dos alunos de 13 e 14 anos

Desafio do Guilherme: aumentar a amostra para alunos menor de idade e compara a proporção por estado.